**Course:** SMM  
**Assignment:** 3

## Team Members
- **Muhammad Rehan** - 02-205252-012  
- **Arzoo Ghani Khan** - 02-205252-003

## Question 1

### Dataset
Raw dataset from: [Kaggle — datascientist97/university-students-marks-sheet](https://www.kaggle.com/datascientist97/university-students-marks-sheet)

# 1. Clean Data and Prepare Final Dataset

In [74]:
# Load Dataset (read 2nd and 3rd sheets)
import pandas as pd

xls = pd.ExcelFile('student_scores_raw.xls')
print("Sheets:", xls.sheet_names)

# read second and third sheets by index (1 and 2)
sessional = pd.read_excel(xls, sheet_name=1)
final = pd.read_excel(xls, sheet_name=2)

print("Sheet 2 preview:")
print(sessional.head())
print("\nSheet 3 preview:")
print(final.head())

Sheets: ['Table', 'Sessional', 'Final']
Sheet 2 preview:
  Roll No           Name Quiz 15  Unnamed: 3 Unnamed: 4 Assignments 15   \
0     NaN            NaN  Quiz 01    Quiz 02   AVG (15)       Assig. 01   
1       1  Ashir Mehfooz       14         14         14              13   
2       2    Atif Raftad        4         10          7               4   
3       3     Saiqa Aziz       15         11         13              14   
4       8   Ozair Minhas        6          5        5.5               4   

  Unnamed: 6 Unnamed: 7 Mid Term    Q+A+Mid  
0   Assig.02   AVG (15)       45   Total 75  
1         13         13       41         68  
2          5        4.5       30       41.5  
3         13       13.5       34       60.5  
4          6          5       12       22.5  

Sheet 3 preview:
  Roll No. Name of Student Total Marks (150)     Unnamed: 3 Unnamed: 4  \
0      NaN             NaN    Sessional (75)  Terminal (75)  Total 150   
1        1   Ashir Mehfooz                68      

In [75]:
# Drop first row of both dataframes
sessional = sessional.drop(index=0).reset_index(drop=True)
final = final.drop(index=0).reset_index(drop=True) 

In [76]:
# Create combined dataframe with specified columns
combined = pd.DataFrame()

# 1. Attendance - set to null
combined['attendance'] = None

# 2. Quizzes (% number - total 15) - 5th column from sessional (index 4)
combined['quizzes'] = pd.to_numeric(sessional.iloc[:, 4], errors='coerce')

# 3. Assignment (% number - total 15) - 8th column from sessional (index 7)
combined['assignment'] = pd.to_numeric(sessional.iloc[:, 7], errors='coerce')

# 4. Final (% number - total 75) - 4rd column from final (index 3)
combined['final'] = pd.to_numeric(final.iloc[:, 3], errors='coerce')

# 5. GPA (0-4 scale) - last column from final
combined['gpa'] = pd.to_numeric(final.iloc[:, -1], errors='coerce')

print("Combined dataframe:")
print(combined.head(10))
print("\nDataframe info:")
print(combined.info())
print("\nDataframe statistics:")
print(combined.describe())

Combined dataframe:
  attendance  quizzes  assignment  final  gpa
0        NaN     14.0        13.0   55.0  4.0
1        NaN      7.0         4.5   45.0  2.5
2        NaN     13.0        13.5   58.0  3.9
3        NaN      5.5         5.0   44.0  0.0
4        NaN      8.0         4.5   35.0  2.2
5        NaN     13.5         3.5   45.0  2.8
6        NaN     12.0        12.5   66.0  NaN
7        NaN      7.5         4.0   43.0  2.6
8        NaN     10.0         5.0   23.0  0.0
9        NaN     14.0         6.0   22.0  0.0

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   attendance  0 non-null      object 
 1   quizzes     216 non-null    float64
 2   assignment  216 non-null    float64
 3   final       213 non-null    float64
 4   gpa         205 non-null    float64
dtypes: float64(4), object(1)
memory usage: 8.9+ KB
None

Datafram

In [77]:
# Check for missing values
print("\nMissing values in each column:")
print(combined.isnull().sum())


Missing values in each column:
attendance    224
quizzes         8
assignment      8
final          11
gpa            19
dtype: int64


In [78]:
# Set the value to max when there is some greater than max
combined.loc[combined['quizzes'] > 15, 'quizzes'] = 15
combined.loc[combined['assignment'] > 15, 'assignment'] = 15
combined.loc[combined['final'] > 75, 'final'] = 75
combined.loc[combined['gpa'] > 4, 'gpa'] = 4

In [79]:
# Except attendance, fill missing values with mean for numerical columns
combined['quizzes'].fillna(combined['quizzes'].mean(), inplace=True)
combined['assignment'].fillna(combined['assignment'].mean(), inplace=True)
combined['final'].fillna(combined['final'].mean(), inplace=True)
combined['gpa'].fillna(combined['gpa'].mean(), inplace=True)
print("\nAfter filling missing values:")
print(combined.isnull().sum())


After filling missing values:
attendance    224
quizzes         0
assignment      0
final           0
gpa             0
dtype: int64


In [80]:
# Set the 0 GPA using quiz score + assignment score + mid(25marks) + final score and calculate GPA

combined.loc[combined["gpa"] == 0, "gpa"] = (
    (combined["quizzes"] + combined["assignment"] + 30 + combined["final"]) / 150 * 4
)

In [81]:
# Range quizzes, assignment, final to 0-100 scale
combined['quizzes'] = ((combined['quizzes'] / 15) * 100).round(2)
combined['assignment'] = ((combined['assignment'] / 15) * 100).round(2)
combined['final'] = ((combined['final'] / 75) * 100).round(2)
combined['gpa'] = combined['gpa'].round(2)

In [82]:
print("Combined dataframe:")
print(combined.head(10))
print("\nDataframe info:")
print(combined.info())
print("\nDataframe statistics:")
print(combined.describe())

Combined dataframe:
  attendance  quizzes  assignment  final   gpa
0        NaN    93.33       86.67  73.33  4.00
1        NaN    46.67       30.00  60.00  2.50
2        NaN    86.67       90.00  77.33  3.90
3        NaN    36.67       33.33  58.67  2.25
4        NaN    53.33       30.00  46.67  2.20
5        NaN    90.00       23.33  60.00  2.80
6        NaN    80.00       83.33  88.00  1.82
7        NaN    50.00       26.67  57.33  2.60
8        NaN    66.67       33.33  30.67  1.81
9        NaN    93.33       40.00  29.33  1.92

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   attendance  0 non-null      object 
 1   quizzes     224 non-null    float64
 2   assignment  224 non-null    float64
 3   final       224 non-null    float64
 4   gpa         224 non-null    float64
dtypes: float64(4), object(1)
memory usage: 8.9+ KB
Non

In [83]:
# Generate realistic attendance based on academic performance
import numpy as np

def generate_attendance(row):
    """
    Generate realistic attendance based on academic performance.
    Students with better grades typically have better attendance.
    
    Base attendance is calculated from normalized performance scores,
    then random variation is added to make it realistic.
    """
    # Normalize scores to 0-1 scale (they're already 0-100)
    quiz_norm = row['quizzes'] / 100
    assignment_norm = row['assignment'] / 100
    final_norm = row['final'] / 100
    gpa_norm = row['gpa'] / 4
    
    # Weighted average (final exam and GPA have more weight)
    performance_score = (quiz_norm * 0.2 + 
                        assignment_norm * 0.2 + 
                        final_norm * 0.3 + 
                        gpa_norm * 0.3)
    
    # Base attendance: 55-95% range based on performance
    # High performers tend to have 80-95% attendance
    # Low performers tend to have 55-75% attendance
    base_attendance = 55 + (performance_score * 40)
    
    # Add random variation (-5 to +5) to make it more realistic
    variation = np.random.uniform(-5, 5)
    attendance = base_attendance + variation
    
    # Ensure it stays within 0-100 range
    attendance = np.clip(attendance, 0, 100)
    
    return round(attendance, 2)

# Set random seed for reproducibility
np.random.seed(42)

# Generate attendance for each student
combined['attendance'] = combined.apply(generate_attendance, axis=1)

print("Updated dataframe with attendance:")
print(combined.head(10))
print("\nAttendance statistics:")
print(combined['attendance'].describe())
print(f"\nAttendance range: {combined['attendance'].min():.2f}% - {combined['attendance'].max():.2f}%")

Updated dataframe with attendance:
   attendance  quizzes  assignment  final   gpa
0       88.95    93.33       86.67  73.33  4.00
1       80.34    46.67       30.00  60.00  2.50
2       92.43    86.67       90.00  77.33  3.90
3       75.38    36.67       33.33  58.67  2.25
4       70.43    53.33       30.00  46.67  2.20
5       76.23    90.00       23.33  60.00  2.80
6       79.67    80.00       83.33  88.00  1.82
7       79.47    50.00       26.67  57.33  2.60
8       73.12    66.67       33.33  30.67  1.81
9       77.03    93.33       40.00  29.33  1.92

Attendance statistics:
count    224.000000
mean      76.601964
std        7.149499
min       59.000000
25%       71.620000
50%       76.510000
75%       81.755000
max       95.040000
Name: attendance, dtype: float64

Attendance range: 59.00% - 95.04%


In [84]:
# Final check for missing values
print("\nMissing values in each column:")
print(combined.isnull().sum())


Missing values in each column:
attendance    0
quizzes       0
assignment    0
final         0
gpa           0
dtype: int64


In [85]:
# Save the cleaned dataset
combined.to_csv('student_scores_clean.csv', index=False)